# Load required modules

In [ ]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

# Set variables

In [ ]:
## Data
COUNT_MATRIX = pd.read_csv('Figure_1/Input_data/GEO_data/10X_SCRNASEQ_WT_EA_AGGR.tsv', sep='\t', header=0, index_col=0).T
TFS_NAMES = load_tf_names('Figure_1/Input_data/SCENIC_data/allTFs_dmel.txt')

## Databases
DATABASE_FOLDER = 'pySCENIC_databases' 
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "dm6-5kb-upstream-full-tx-11species.mc8nr.feather") # Feather databases are available at: https://resources.aertslab.org/cistarget/
MOTIF_ANNOTATIONS_FNAME = os.path.join(DATABASE_FOLDER, "motifs-v8-nr.flybase-m0.001-o0.0.tbl") # Available at: https://resources.aertslab.org/cistarget/motif2tf/

## Load databases
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]

## Output folder
OUT_FOLDER="Figure_1/Processed_data/SCENIC/"
ADJACENCIES_FNAME = os.path.join(OUT_FOLDER, "adjacencies.tsv")
MODULES_FNAME = os.path.join(OUT_FOLDER, "modules.p")
MOTIFS_FNAME = os.path.join(OUT_FOLDER, "motifs.p")
REGULONS_FNAME = os.path.join(OUT_FOLDER, "regulons.p")
AUCMAT_FNAME = os.path.join(OUT_FOLDER, "aucMatrix.tsv")

## Set scheduler
from distributed import LocalCluster, Client
SCHEDULER = Client('tcp://10.118.224.141:8786')

# Infer co-expression modules

In [ ]:
adjacencies = grnboost2(expression_data=COUNT_MATRIX, tf_names=TFS_NAMES, seed=123, client_or_address=SCHEDULER)
adjacencies.to_csv(ADJACENCIES_FNAME, index=False, sep='\t')
modules = list(modules_from_adjacencies(adjacencies, COUNT_MATRIX))
with open(MODULES_FNAME, 'wb') as f:
    pickle.dump(modules, f)

# Prune modules based for targets with cis-regulatory footprints (cisTarget)

In [ ]:
# Motif enrichment
motifEnr = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address=SCHEDULER)
with open(MOTIFS_FNAME, 'wb') as f:
    pickle.dump(motifEnr, f)

# Cleanup and export to text
def export_motifs2txt(motifEnr, fileName):
    fileName=os.path.splitext(fileName)[0]+'.tsv' 
    met=motifEnr['Enrichment']
    met.Context = [list(dbn)[2] for dbn in met.Context]
    met.TargetGenes=["; ".join(sorted([gs[0] for gs in row])) for row in met.TargetGenes]
    met=met.drop(columns='Annotation')
    met.columns.values[met.columns.values=='Context']='DB'
    met.to_csv(fileName, sep='\t') 
            
export_motifs2txt(motifEnr, MOTIFS_FNAME)

regulons = df2regulons(motifEnr)
with open(REGULONS_FNAME, 'wb') as f:
    pickle.dump(regulons, f)
    
regulons_original=regulons
regulons = [r.rename(r.name.replace('(+)',' ('+str(len(r))+'g)')) for r in regulons]

# Export (e.g. to import in R SCENIC)
def export_asGmt(regulons, fileName):
    fileName=os.path.splitext(fileName)[0]+'.gmt'
    with open(fileName, 'w') as f:
        for reg in regulons:
            f.write(reg.name + "\t.\t" + '\t'.join(list(reg.gene2weight.keys())) + '\n')
            

export_asGmt(modules, MODULES_FNAME)
export_asGmt(regulons, REGULONS_FNAME)

# Cellular regulon enrichment matrix (AUCell)

In [ ]:
auc_mtx = aucell(COUNT_MATRIX, regulons, num_workers=1)
auc_mtx = auc_mtx.loc[ex_matrix.index]
auc_mtx.to_csv(AUCMAT_FNAME) 

# ChIP-seq regulons

In [ ]:
## Databases
DATABASE_FOLDER = 'Figure_1/Input_data/SCENIC_data/' 
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "encode_modERN_20190621__ChIP_seq.max_GENEBASED.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(DATABASE_FOLDER, "encode_modERN_20190621_dm6_annotation.tbl")

## Load databases
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
    
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]

## Load modules
with open(MODULES_FNAME, 'rb') as f:
    modules = pickle.load(f)

# Motif enrichment
MOTIFS_FNAME = os.path.join(OUT_FOLDER, "motifs_ChIPseq.p")
motifEnr = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME)
with open(MOTIFS_FNAME, 'wb') as f:
    pickle.dump(motifEnr, f)

# Cleanup and export to text
def export_motifs2txt(motifEnr, fileName):
    fileName=os.path.splitext(fileName)[0]+'.tsv' 
    met=motifEnr['Enrichment']
    met.Context = [list(dbn)[2] for dbn in met.Context]
    met.TargetGenes=["; ".join(sorted([gs[0] for gs in row])) for row in met.TargetGenes]
    met=met.drop(columns='Annotation')
    met.columns.values[met.columns.values=='Context']='DB'
    met.to_csv(fileName, sep='\t') 
            
export_motifs2txt(motifEnr, MOTIFS_FNAME)

REGULONS_FNAME = os.path.join(OUT_FOLDER, "regulons_ChIPseq.p")
regulons = df2regulons(motifEnr)
with open(REGULONS_FNAME, 'wb') as f:
    pickle.dump(regulons, f)
    
regulons_original=regulons
regulons = [r.rename(r.name.replace('(+)',' ('+str(len(r))+'g)')) for r in regulons]

# Export (e.g. to import in R SCENIC)
def export_asGmt(regulons, fileName):
    fileName=os.path.splitext(fileName)[0]+'.gmt'
    with open(fileName, 'w') as f:
        for reg in regulons:
            f.write(reg.name + "\t.\t" + '\t'.join(list(reg.gene2weight.keys())) + '\n')
            

export_asGmt(modules, MODULES_FNAME)
export_asGmt(regulons, REGULONS_FNAME)

# AUCell
AUCMAT_FNAME = os.path.join(OUT_FOLDER, "aucMatrix_ChIPseq.tsv")
auc_mtx = aucell(COUNT_MATRIX, regulons, num_workers=1)
auc_mtx = auc_mtx.loc[ex_matrix.index]
auc_mtx.to_csv(AUCMAT_FNAME) 

For further details on how to run pySCENIC, please visit: https://github.com/aertslab/pySCENIC